In [1]:
import requests,pandas as pd, sys
from bs4 import BeautifulSoup

df_overview_full = pd.DataFrame()
df_years_full = pd.DataFrame()
df_each_full = pd.DataFrame()

current_path = sys.path[0]

url = 'https://www.dohod.ru/ik/analytics/dividend'

In [2]:
### Составление общего списка
def get_main (url):
    
    html = requests.get(url).content
    df_list = pd.read_html(html)
    df_mainpage = df_list[-1]

    # очистка от ненужных колонок
    bad_columns = []
    for i in list(df_mainpage.columns):
        if "Unnamed:" in i or "Капитализация, sorting" in i:
            bad_columns.append(i)

    # print(bad_columns)
    df_mainpage.drop(columns=bad_columns, inplace=True)

    # len(df_mainpage)
    if len(df_mainpage) > 0:
        df_mainpage.to_excel(current_path +'/obzor_vsex_divs_tickerov.xlsx')
    else:
        print('Не удалось скачать актуальные данные о дивах с доход.ру')
    return df_mainpage

In [3]:
def get_page_info (url):

    df_overview_full = pd.DataFrame()
    df_years_full = pd.DataFrame()
    df_each_full = pd.DataFrame()

    ### Сбор всех ссылок на страницы
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'lxml')

    data = []

    for link in soup.find_all('a', href=True):
        data.append(str(link.get('href')))


    data = list(set(data))

    url_list = []

    for i in data:
        if "/ik/analytics/dividend/" in i and ".pdf" not in i:
            url = "https://www.dohod.ru" + i
            url_list.append(url)
    

    print("Количество страниц для обхода: {}".format(len(url_list)))


    ### Обход каждой страницы

    b = []
    for url in url_list:
        html = requests.get(url).content
        try:
            df_list = pd.read_html(html)
        except:
            df_list = pd.DataFrame()
            print('Не смог найти таблицы на этой странице: {}',format(url))

        a = len(df_list)
        b.append(a)

        if len(df_list) == 2:
            df = df_list[0]
            df.columns = df.iloc[1]
            df.reset_index(inplace=True)
            df = df.drop (index= 1)
            df = df[['текущая доходность','доля от прибыли','индекс DSI']]
            df['page_url'] = url
            df['ticker'] = url.split(r'/')[-1].upper()
            df_overview_full = pd.concat([df_overview_full,df])

            df = df_list[1]
            df['page_url'] = url
            df['ticker'] = url.split(r'/')[-1].upper()
            df_years_full = pd.concat([df_years_full,df])

        elif len(df_list) == 3:
            df = df_list[0]
            df.columns = df.iloc[1]
            df.reset_index(inplace=True)
            df = df.drop (index= 1)
            df = df[['текущая доходность','доля от прибыли','индекс DSI']]
            df['page_url'] = url
            df['ticker'] = url.split(r'/')[-1].upper()
            df_overview_full = pd.concat([df_overview_full,df])

            df = df_list[1]
            df['page_url'] = url
            df['ticker'] = url.split(r'/')[-1].upper()
            df_years_full = pd.concat([df_years_full,df])

            df = df_list[2]
            df['page_url'] = url
            df['ticker'] = url.split(r'/')[-1].upper()
            df_each_full = pd.concat([df_each_full,df])


    ## Статистика для отладки. Сколько разных табличек спарсено
    o = list(set(b))

    for i in o:
        print(i)
        print(b.count(i))
        print("___")
    
    return df_overview_full, df_years_full, df_each_full

In [4]:
df_mainpage = get_main(url)
df_mainpage

,Акция,Сектор,Период,Выплата на акцию (прогноз),Доходность выплаты,Цена расчета доходности,Дата закрытия реестра (оценка),"Капитализация, млн. USD",DSI
0,Fix Price,Розничная торговля,всего 12м,53.53,20.11%,266.20,NaN,62 239.79,0.07
1,Сургутнефтегаз-п,Нефть/Газ,годовые 2023,12.29,18.29%,67.18,18.07.2024,1 565 085.04,0.50
2,МКПАО ЮМГ (ЕМС),Разное,всего 12м,129.76,17.91%,724.50,NaN,65 205.00,0.00
3,НЛМК,Металлы и добыча,всего 12м,31.40,17.7%,177.46,NaN,1 063 558.11,0.07
4,Совкомфлот,Нефть/Газ,всего 12м,19.22,15.88%,121.06,NaN,287 516.76,0.14
...,...,...,...,...,...,...,...,...,...
295,Юнипро,Энергетика,годовые 2024,0.00,0%,1.93,13.05.2025,121 368.76,0.00
296,НКНХ-ао,Химия,I полугодие 2024,0.00,0%,94.10,05.12.2024,167 609.38,0.50
297,ТКС Холдинг,Финансы и Банки,I квартал 2024,0.00,0%,2976.00,21.08.2024,593 133.14,0.00
298,Эталон,Строительство,годовые 2023,0.00,0%,82.76,08.07.2024,24 186.78,0.00


In [5]:
df_overview_full, df_years_full, df_each_full = get_page_info (url)
df_overview_full.to_excel(current_path +'/rate_for_each_ticker.xlsx')
df_overview_full.to_csv(current_path +'/rate_for_each_ticker.csv')

df_years_full.to_excel(current_path +'/sum_each_year_divs.xlsx')
df_years_full.to_csv(current_path +'/sum_each_year_divs.csv')

df_each_full.to_excel(current_path +'/all_payments.xlsx')
df_each_full.to_csv(current_path +'/all_payments.csv')

Количество страниц для обхода: 119
2
8
___
3
111
___
